You can download and run this notebook locally, or you can run it for free in a cloud environment using Colab or Sagemaker Studio Lab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SL2027/TCIA_Notebooks/blob/main/TCIA_REST_API_Downloads.ipynb)

[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github.com/SL2027/TCIA_Notebooks/blob/main/TCIA_REST_API_Downloads.ipynb)

# Summary

Access to large, high quality data is essential for researchers to understand disease and precision medicine pathways, especially in cancer. However HIPAA constraints make sharing medical images outside an individual institution a complex process. [The Cancer Imaging Archive (TCIA)](https://www.cancerimagingarchive.net/) is a public service funded by the National Cancer Institute which addresses this challenge by providing hosting and de-identification services to take major burdens of data sharing off researchers. 

**This notebook is focused on basic use cases for leveraging TCIA's REST APIs to query and download data.**  If you're interested in additional TCIA notebooks and coding examples check out https://github.com/kirbyju/TCIA_Notebooks. 

# 1 Learn about available Collections on the TCIA website

[Browsing Collections](https://www.cancerimagingarchive.net/collections) and [Analysis Results](https://www.cancerimagingarchive.net/tcia-analysis-results/) datasets on TCIA are the easiest ways to become familiar with what is available.  These pages will help you quickly identify datasets of interest, find valuable supporting data that are not available via our APIs (e.g. clinical spreadsheets, non-DICOM segmentation data), and answer most common questions you might have about the datasets.  

# 2 REST API Overview 
TCIA uses software called NBIA to manage DICOM data.  The NBIA REST APIs include:
1. [NBIA Search REST APIs](https://wiki.cancerimagingarchive.net/x/fILTB) that allow you to perform basic queries and download data from **public** collections. This API does not require a TCIA account.
2. [NBIA Search with Authentication REST APIs](https://wiki.cancerimagingarchive.net/x/X4ATBg) that allow you to perform basic queries and download data from **public and limited-access** collections. This API requires a TCIA account for creation of authentication tokens.
3. [NBIA Advanced REST APIs](https://wiki.cancerimagingarchive.net/x/YoATBg) that allow access to **public and limited-access** collections, but provide query endpoints mostly geared towards developers seeking to integrate searching and downloading TCIA data into web and desktop applications.  This API requires a TCIA account for creation of authentication tokens.

# 3 Import tcia_utils

The following cells import [**tcia_utils**](https://pypi.org/project/tcia-utils/) which contain a variety of useful functions for accessing TCIA via Jupyter/Python. 

**tcia_utils** contains a **downloadSeries()** function that has multiple options for specifying the seriesUids you'd like to download.  By default, the function expects JSON data containing "SeriesInstanceUID" elements, which can be generated using **getSeries()** or **getCart()**.  However, if you have a series UID list from some other source, you can set **input_type = "list"** to pass a python list of one or more series UIDs instead of JSON. You can also set **input_type = "manifest"** and provide a path to a **.TCIA** manifest file.

Data are saved to a **tciaDownload** folder in your current working directory by default, but you can use the **path** parameter to change this to a different directory.

There is an optional **format** parameter that can be used to return metadata about what was downloaded.  It can be set to **df** to return dataframe or **csv** to save a spreadsheet. There's also a **csv_filename** parameter if you want to set a specific file name.

You can specify **number = n** to tell the function to only download the first **n** scans of your seriesUids.  Remove this parameter in the examples below to download the full dataset. 

The **api_url** parameter can be omitted in most cases.  However, it must be set to **api_url = "nlst"** to access the [National Lung Screening Trial (NLST)](https://doi.org/10.7937/TCIA.HMQ8-J677) collection and you must use **api_url = "restricted"** for datasets that require logging in.  

Last but not least, there is some logic built in to detect whether you've already downloaded a series.  If a directory named after the seriesUid already exists the function will assume it's already been downloaded and skip that series.

In [ ]:
!pip install --upgrade -q tcia-utils
!pip install --upgrade -q pandas

In [ ]:
from tcia_utils import nbia
import pandas as pd
import requests

### Set logging level to INFO in Google Colab (optional)
This step should be skipped unless you're running on **Google Colab** as its logging root handler only shows warnings and errors by default.  If you'd like to see INFO statements you can run the following code.  This is particularly helpful when running some of the download examples so you can see the progress as downloads complete.

In [ ]:
import logging

# Check current handlers
#print(logging.root.handlers)

# Remove all handlers associated with the root logger object.
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
#print(logging.root.handlers)

# Set handler with level = info
logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s', 
                    level=logging.INFO)

print("Logging set to INFO")

# 4 Download Examples

In this section we'll cover downloading data via the REST API for the following use cases:

1.   Download a full TCIA collection
2.   Download custom results of an API query
3.   Download a "[Shared Cart](https://wiki.cancerimagingarchive.net/display/NBIA/TCIA+Radiology+Portal+User+Guide#TCIARadiologyPortalUserGuide-SharingDatainYourCart)" that was created via https://nbia.cancerimagingarchive.net/
4.   Download data from a TCIA manifest file
5.   Download data from a **restricted** collection that requires creating an API token
6.   Download images

## 4.1 Download a full collection

You can [Browse Collections](https://www.cancerimagingarchive.net/collections) on our website to figure out what you might want to download, but you can also get a list of available collections via the API as shown below.

In [ ]:
# get list of available collections as JSON
nbia.getCollections()


Let's say that we're interested in downloading the entire **Soft-tissue-Sarcoma** collection.  First we need to get a list of all Series Instance UIDs in that collection.  We can use **nbia.getSeries()** to return JSON metadata about all series (scans) in this collection.

In [ ]:
data = nbia.getSeries(collection = "Soft-tissue-Sarcoma")
print(data)

We can then pass that to the our download functions and view/save the metadata for what was downloaded.  We'll leverage the **number** parameter here to just grab the first 3 scans as a test.  You can remove this parameter if you want to download the full collection.

In [ ]:
nbia.downloadSeries(data, number = 3)

You can learn more about various ways to visualize and analyze your data in the other notebooks at https://github.com/kirbyju/TCIA_Notebooks.  However, let's use **nbia.viewSeries()** to get a quick look at one of the series we've downloaded.  You can change the Series UID in the code below to view other scans you've downloaded.

In [ ]:
seriesUid = "1.3.6.1.4.1.14519.5.2.1.5168.1900.104193299251798317056218297018"
nbia.viewSeries(seriesUid)

## 4.2 Download custom API query
The REST API allows for a variety of different query options as demonstrated in [this notebook](https://github.com/kirbyju/TCIA_Notebooks/blob/main/TCIA_REST_API_Queries.ipynb).  For this use case, let's assume that you are only interested in the MR scans from the [TCGA-BRCA](https://doi.org/10.7937/K9/TCIA.2016.AB2NAZRP) collection that were acquired on Siemens scanners.

In [ ]:
# getSeries with query parameters
data = nbia.getSeries(collection = "TCGA-BRCA", 
               modality = "MR", 
               manufacturer = "SIEMENS")

print(len(data), 'Series returned')

Once again, let's pass those Series Instance UIDs to our download function.  This time we'll also set **format = "df"** to return a dataframe about what we downloaded.

In [ ]:
# feed series_data to our downloadSampleSeries function
df = nbia.downloadSeries(data, number = 3, format = "df")
display(df)

Another common use case may be that you'd like to review the results from **getSeries()** and do some further filtering to your dataframe.  Then, after removing unwanted scans, pass the remaining Series Instance UIDs to **downloadSeries()**.  For example, let's say that you only wanted to download **T2 MR** series instead of any MR.

In [ ]:
# getSeries with query parameters
df = nbia.getSeries(collection = "TCGA-BRCA", 
               modality = "MR", 
               manufacturer = "SIEMENS", format = "df")

Let's drop scans that don't contain **t2** in either the **ProtocolName** or **SeriesDescription** fields.  Note that we're down to 21 scans now instead of 353.

In [ ]:
# convert the columns to lowercase to catch both upper/lower case with filter for 't2'
filtered_df = df[(df['ProtocolName'].str.lower().str.contains('t2')) | 
                 (df['SeriesDescription'].str.lower().str.contains('t2'))]

display(filtered_df)

Now we can feed the Series Instance UIDs from this dataframe back to **downloadSeries()**.  Note that since we're not working with the API's default JSON output anymore we need to extract the Series UIDs from the dataframe into a **list** object and specify **input_type = "list"** when we call **downloadSeries()**.

In [ ]:
# extract the SeriesInstanceUID column as a list variable
series_uids = filtered_df['SeriesInstanceUID'].tolist()

# download the selected series_uids
nbia.downloadSeries(series_uids, input_type = "list", number = 3)

## 4.3 Download custom NLST API query
Let's show a similar example where we look for a specific modality and manufacturer within the [National Lung Screening Trial (NLST) Collection](https://doi.org/10.7937/TCIA.HMQ8-J677).  We have to set **api_url = "nlst"** in our functions for this to work, but otherwise the steps are the same.

In [ ]:
# getSeries with query parameters
data = nbia.getSeries(collection = "NLST", 
               modality = "CT", 
               manufacturer = "Philips",
               api_url = "nlst")

print(len(data), 'Series returned')

In [ ]:
# feed series_data to our downloadSampleSeries function
df = nbia.downloadSeries(data, number = 3, api_url = "nlst", format = "df")
display(df)

## 4.4 Download a "shared cart"
It's possible to use https://nbia.cancerimagingarchive.net to create a "[Shared Cart](https://wiki.cancerimagingarchive.net/display/NBIA/TCIA+Radiology+Portal+User+Guide#TCIARadiologyPortalUserGuide-SharingDatainYourCart)" which includes a specific set of scans you'd like to share with others. After creating a Shared Cart you receive a URL like https://nbia.cancerimagingarchive.net/nbia-search/?saved-cart=nbia-49121659384603347 which can be shared with others.  Try clicking the link to see what this looks like on the TCIA website.  Then use the code below to see how you can use the cart name at the end of the URL to download the related scans via the API.

In [ ]:
# getSharedCart metadata
data = nbia.getSharedCart(name = "nbia-49121659384603347")
print(len(data), 'Series returned')

We'll skip the use of the **number** parameter this time since the full cart is only 4 series.  Let's also try **format = "csv"** to save a spreadsheet of the metadata in addition to returning a dataframe.

In [ ]:
# feed series_data to our downloadSampleSeries function
df = nbia.downloadSeries(data, format = "csv")
display(df)

## 4.4 Download data from a TCIA manifest file

When working with manifest files you can install the NBIA Data Retriever to open the manifest and download the data as shown in [this notebook](https://github.com/kirbyju/TCIA_Notebooks/blob/main/TCIA_Linux_Data_Retriever_App.ipynb).  However, there may be cases where you don't have administrative rights to install software or prefer using the REST API to download a manifest.  

In order to demonstrate this use case, let's assume that after you [Browse Collections](https://www.cancerimagingarchive.net/collections) you are interested in the [RIDER Breast MRI](https://doi.org/10.7937/K9/TCIA.2015.H1SXNUXL) collection.  We can find the URL of the manifest to download the full collection by looking at the blue "Download" button on that page.  Then we can download the manifest with the following commands.  

In [ ]:
# download manifest file from RIDER Breast MRI page
manifest = requests.get("https://wiki.cancerimagingarchive.net/download/attachments/22512757/doiJNLP-Fo0H1NtD.tcia?version=1&modificationDate=1534787017928&api=v2")
with open('RIDER_Breast_MRI.tcia', 'wb') as f:
    f.write(manifest.content)

If you open this manifest file in a text editor you'll notice that it contains several lines of download parameters that precede a list of Series Instance UIDs to download.  If we set **input_type = "manifest"** we can provide the path/filename to **downloadSeries()** and it will extract the UIDs from the file and download them.

In [ ]:
df = nbia.downloadSeries("RIDER_Breast_MRI.tcia", input_type = "manifest", number = 3, format = "df")
display(df)

## 4.5 Download data from a restricted collection
In some cases, you must specifically request access to collections before you can download them.  These are listed as **limited access** on the [Browse Collections](https://www.cancerimagingarchive.net/collections/) page. The steps to request access may vary depending on the collection, but will always require that you first [create a TCIA user account](https://wiki.cancerimagingarchive.net/x/xgHDAg). Once you've created an account, you can use your login/password to create an API token with the **getToken()** function from **tcia_utils** to verify your permissions. **<font color='red'>Tokens are valid for 2 hours and must be refreshed after that point.</font>**

In [ ]:
nbia.getToken()

Let's say that we're interested in the [QIN-Breast-02](https://doi.org/10.7937/TCIA.2019.4cfm06rr) collection. As you can see on the collection page, you must email help@cancerimagingarchive.net to request access to the data. Once you've recieved approval we can use **nbia.getSeries()** to get a full list of series UIDs in this restricted collection by including **api_url = "restricted"** as a parameter.

In [ ]:
# getSeries with query parameters
data = nbia.getSeries(collection = "QIN-Breast-02", 
                      api_url = "restricted")

print(len(data), 'Series returned')

Now we can download those scans.  Don't forget to include **api_url = "restricted"** in the download functions as well!

In [ ]:
# feed series_data to our downloadSampleSeries function
df = nbia.downloadSeries(data, number = 3, api_url = "restricted", format = "df")
display(df)

## 4.6 Download Images

You can [Browse Collections](https://www.cancerimagingarchive.net/collections) on our website to figure out what you might want to download, but you can also get a list of available collections via the API as shown below.

In [ ]:
# get list of available collections as JSON
nbia.getCollections()

Let's say that we're interested in downloading the images associated with the **CMB-GEC** collection.  First we need to get a list of all Series Instance UIDs in that collection.  We can use **nbia.getSeries()** to return JSON metadata about all series (scans) in this collection.

In [ ]:
nbia.getSeries(collection = "CMB-GEC")

We can then pass that to the our **nbia.getSopInstanceUids()** functions and view a list of metadata associated with the series and find the SOP instance we want to download the images from.

In [ ]:
nbia.getSopInstanceUids(seriesUid = "1.3.6.1.4.1.14519.5.2.1.1600.1204.919741553251398079475267746505")

After we have both the series UID and SOP UID, we can call the **nbia.downloadImage()** function to download all images associated with the instance. Note that when the path parameter is not provided, a folder tciaDownload will be automatically created.

In [ ]:
nbia.downloadImage(seriesUID = "1.3.6.1.4.1.14519.5.2.1.1600.1204.919741553251398079475267746505", 
              sopUID = "1.3.6.1.4.1.14519.5.2.1.1600.1204.211684247543622814130853101548")

# Acknowledgements
TCIA is funded by the [Cancer Imaging Program (CIP)](https://imaging.cancer.gov/), a part of the United States [National Cancer Institute (NCI)](https://www.cancer.gov/).  It is managed by the [Frederick National Laboratory for Cancer Research (FNLCR)](https://frederick.cancer.gov/) and hosted by the [University of Arkansas for Medical Sciences (UAMS)](https://www.uams.edu/)

This notebook was created by [Justin Kirby](https://www.linkedin.com/in/justinkirby82/). If you leverage this notebook or any TCIA datasets in your work, please be sure to comply with the [TCIA Data Usage Policy](https://wiki.cancerimagingarchive.net/x/c4hF). In particular, make sure to cite the DOI(s) for the specific TCIA datasets you used in addition to the following paper!

## TCIA Citation

Clark, K., Vendt, B., Smith, K., Freymann, J., Kirby, J., Koppel, P., Moore, S., Phillips, S., Maffitt, D., Pringle, M., Tarbox, L., & Prior, F. (2013). The Cancer Imaging Archive (TCIA): Maintaining and Operating a Public Information Repository. Journal of Digital Imaging, 26(6), 1045–1057. https://doi.org/10.1007/s10278-013-9622-7